# CDS of enriched flagellins across biomes

This notebook will focus on phylogenetic analyses with differentially abundant (DA) flagellins previously identified between host-associated and free-living environments

### Set-up

In [2]:
library(tidyverse)
library(dplyr)
library(readr)
library(stringr)
library(ggplot2)
library(ggsci)
library(viridis)
library(conflicted)
library(gridExtra)
library(vegan)
library(ape)
library(lattice)
library(permute)
library(grid) 
library(seqinr)
library("phytools")
library(rentrez)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.0.9     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.4.1
✔ ggplot2   3.3.6     ✔ tibble    3.2.1
✔ lubridate 1.8.0     ✔ tidyr     1.2.1
✔ purrr     0.3.5     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: viridisLite

Loading required package: permute

Loading required package: lattice

This is vegan 2.6-4

Loading required package: maps


Attaching package: ‘maps’


The following object is masked from ‘package:viridis’:

    unemp


The following object is masked from ‘pack

In [3]:
getwd()

[1] "/ebio/abt3_projects2/Flagellin_Diversity/code/notebooks/shortbred/edgeR/subsampled_opt1/cds"

In [5]:
conflict_prefer("filter","dplyr")
conflict_prefer("rename","dplyr")
conflict_prefer("mutate","dplyr")
conflict_prefer("count","dplyr")
conflict_prefer("read.fasta","seqinr")

[conflicted] Will prefer dplyr::filter over any other package.
[conflicted] Will prefer dplyr::rename over any other package.
[conflicted] Will prefer dplyr::mutate over any other package.
[conflicted] Will prefer dplyr::count over any other package.
[conflicted] Will prefer seqinr::read.fasta over any other package.


In [191]:
#Import miscellaneous functions for plots and edger
source("/ebio/abt3_projects2/Flagellin_Diversity/code/notebooks/FlagellinDiversity/plots_functions.r")

# Get protein-matching CDS (coding sequences): Batch Entrez

bash code for batch entrez

### Import list of DA flagellins in each biome

In [8]:
getwd()

[1] "/ebio/abt3_projects2/Flagellin_Diversity/code/notebooks/shortbred/edgeR/subsampled_opt1/cds"

In [6]:
DA.animals=read_tsv("DA.animals.tsv",col_names=T)
DA.plants=read_tsv("DA.plants.tsv",col_names=T)
DA.env=read_tsv("DA.env.tsv",col_names=T)
DA.host=read_tsv("DA.host.tsv",col_names=T)

Rows: 121 Columns: 13
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (9): Accession, Domain, Phylum, Class, Order, Family, Genus, Species, En...
dbl (4): logFC, logCPM, PValue, FDR

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 82 Columns: 13
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (9): Accession, Domain, Phylum, Class, Order, Family, Genus, Species, En...
dbl (4): logFC, logCPM, PValue, FDR

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 513 Columns: 13
── Column specification

In [7]:
DA.all.biomes = rbind(DA.animals,DA.plants,DA.env,DA.host)

### Import cds fasta files

In [4]:
#Import fasta files with cds obtained with Batch entrez
cds.animals = read.fasta(file = "cds_animals.fna")
cds.plants = read.fasta(file = "cds_plants.fna")
cds.env = read.fasta(file="cds_env.fna")
cds.host = read.fasta(file="cds_host.fna")

In [8]:
cds.table = bind_cols(enriched_in=c("Animals","Plants","Environmental","Host"),
                     No.DAs=c(nrow(DA.animals),
                              nrow(DA.plants),
                              nrow(DA.env),
                              nrow(DA.host)))

cds.table

enriched_in,No.DAs
<chr>,<int>
Animals,121
Plants,82
Environmental,513
Host,110


In [22]:
length(intersect(DA.animals$Accession,DA.host$Accession))
length(intersect(DA.plants$Accession,DA.host$Accession))

[1] 94

[1] 11

In [9]:
files.list=list.files(path="/ebio/abt3_projects2/Flagellin_Diversity/code/notebooks/shortbred/edgeR/subsampled_opt1/cds",pattern="*.fna")
#df_list <- lapply(files.list,read.fasta)

#### Functions to get sequence names and obtain their protein accession from header

In [35]:
seqNames.animals = getSeqNames(cds.animals)
seqNames.plants = getSeqNames(cds.plants)
seqNames.env = getSeqNames(cds.env)
seqNames.host = getSeqNames(cds.host)

Warning message:
“Expected 3 pieces. Additional pieces discarded in 87 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”
Warning message:
“Expected 3 pieces. Additional pieces discarded in 52 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”
Warning message:
“Expected 3 pieces. Additional pieces discarded in 317 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”
Warning message:
“Expected 3 pieces. Additional pieces discarded in 81 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


In [36]:
head(seqNames.plants)

seqName,cds_name,type,Accession
<chr>,<chr>,<chr>,<chr>
lcl|CU234118.1_cds_CAL78604.1_1,lcl|CU234118.1,cds,CAL78604.1
lcl|CP001074.1_cds_ACE89738.1_1,lcl|CP001074.1,cds,ACE89738.1
lcl|CP001074.1_cds_ACE93333.1_1,lcl|CP001074.1,cds,ACE93333.1
lcl|ACKW01000055.1_cds_EEJ52905.1_1,lcl|ACKW01000055.1,cds,EEJ52905.1
lcl|GU071047.1_cds_ACY40759.1_1,lcl|GU071047.1,cds,ACY40759.1
lcl|HE617161.1_cds_CCF11937.1_1,lcl|HE617161.1,cds,CCF11937.1


#### Find remaining sequences without corresponding CDS. Write the lists

In [37]:
remaining.cds.animals = anti_join(DA.animals,seqNames.animals,by="Accession")
remaining.cds.plants = anti_join(DA.plants,seqNames.plants,by="Accession")
remaining.cds.env = anti_join(DA.env,seqNames.env,by="Accession")
remaining.cds.host =anti_join(DA.host,seqNames.host,by="Accession")

In [38]:
#Write remaining
write_tsv(remaining.cds.animals,"rem.cds.animals.tsv")
write_tsv(remaining.cds.plants,"rem.cds.plants.tsv")
write_tsv(remaining.cds.env,"rem.cds.env.tsv")
write_tsv(remaining.cds.host,"rem.cds.host.tsv")

#### List remaining cds for each dataset

In [39]:
rem.cds.table = bind_cols(enriched_in=c("Animals","Plants","Environmental","Host"),
                     No.DAs=c(nrow(remaining.cds.animals),
                              nrow(remaining.cds.plants),
                              nrow(remaining.cds.env),
                              nrow(remaining.cds.host)))

rem.cds.table

enriched_in,No.DAs
<chr>,<int>
Animals,34
Plants,30
Environmental,196
Host,29


In [12]:
#This function does an additional step to remove last characters from the accession number of the proteins.
getSeqNames.rem = function(fasta_file){
#     seq_names = getName(fasta_file)
    #Split the sequence names into three columns based on underscores
            seq_file = read.fasta(fasta_file)    
            seq_table = tibble(seqName=getName(seq_file))%>%
                      separate(col=seqName,into=c("nuc","Protein"),sep="_cds_",remove="FALSE")%>%
                      mutate(Accession = str_replace_all(Protein,"_[^_]*$", "")) #Removes from last position until underscore
    return(seq_table)
    }

# CDS - filter and merge

### A. Animals-enriched cds

#### Get names of remaining CDS

In [10]:
seqs.rem.cds.animals=read.fasta("rem.cds.animals.fna")

In [11]:
names.rem.animals = getSeqNames.rem("rem.cds.animals.fna")

ERROR: Error in getSeqNames.rem("rem.cds.animals.fna"): could not find function "getSeqNames.rem"


In [ ]:
head(names.rem.animals)

In [ ]:
ToKeep.cds = filter(names.rem.animals,Accession%in%remaining.cds.animals$Accession)

In [ ]:
last = anti_join(remaining.cds.animals,ToKeep.cds,by="Accession")
last

#### Filter CDS of matching flagellins 

In [ ]:
keep.cds.animals = seqs.rem.cds.animals[c(which(names(seqs.rem.cds.animals) %in% ToKeep.cds$seqName))]

In [ ]:
length(keep.cds.animals)

In [ ]:
#Merge all CDS files
all.cds.animals = c(cds.animals,keep.cds.animals)

In [50]:
#Write fasta with all cds 
write.fasta(sequences=all.cds.animals,names=names(all.cds.animals),nbchar=80,file.out="all.cds.animals.fna")

### B. Plants-enriched flagellins

In [52]:
seqs.rem.plants = read.fasta("rem.cds.plants.fna")

In [53]:
names.rem.plants = getSeqNames.rem("rem.cds.plants.fna")

In [54]:
toKeep.cds.plants = filter(names.rem.plants,Accession%in%remaining.cds.plants$Accession)

In [55]:
obsolete.plants = anti_join(remaining.cds.plants,toKeep.cds.plants,by="Accession")
obsolete.plants

Accession,Domain,Phylum,Class,Order,Family,Genus,Species,logFC,logCPM,PValue,FDR,EnrichedIn
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
WP_109855741.1,d__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__Rhizobiales,f__Rhizobiaceae,g__Ensifer,s__Ensifer adhaerens,1.6244679,10.60927,1.297648e-19,2.322789e-17,Plants
WP_082564463.1,d__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__Caulobacterales,f__Caulobacteraceae,g__Caulobacter,s__Caulobacter rhizosphaerae,0.9055468,10.44234,4.267642e-09,1.556109e-07,Plants
WP_071836315.1,d__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__Rhizobiales,f__Rhizobiaceae,g__Pararhizobium,s__Pararhizobium antarcticum,0.8894392,10.44252,9.762742e-09,3.258108e-07,Plants
WP_112429052.1,d__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__Rhizobiales,f__Rhizobiaceae,g__Rhizobium,s__Rhizobium sp900472605,0.6035697,10.39848,1.457005e-04,1.827289e-03,Plants
WP_066854468.1,d__Bacteria,p__Desulfobacterota,c__Desulfovibrionia,o__Desulfovibrionales,f__Desulfovibrionaceae,g__Halodesulfovibrio,s__Halodesulfovibrio spirochaetisodalis,0.5436066,10.40908,5.970899e-04,6.220476e-03,Plants


#### Filter CDS of matching flagellins 

In [56]:
keep.cds.plants = seqs.rem.plants[c(which(names(seqs.rem.plants) %in% toKeep.cds.plants$seqName))]

In [57]:
length(keep.cds.plants)

[1] 25

In [58]:
#Merge all CDS files
all.cds.plants = c(cds.plants,keep.cds.plants)

In [59]:
#Write fasta with all cds 
write.fasta(sequences=all.cds.plants,names=names(all.cds.plants),nbchar=80,file.out="all.cds.plants.fna")

### C. Host-enriched CDS

In [61]:
seqs.rem.cds.host=read.fasta("rem.cds.host.fna")

In [62]:
names.rem.host = getSeqNames.rem("rem.cds.host.fna")

In [63]:
head(names.rem.host)

seqName,nuc,Protein,Accession
<chr>,<chr>,<chr>,<chr>
lcl|NZ_LOON01000008.1_cds_WP_000997010.1_1,lcl|NZ_LOON01000008.1,WP_000997010.1_1,WP_000997010.1
lcl|NZ_LOON01000008.1_cds_WP_000648572.1_2,lcl|NZ_LOON01000008.1,WP_000648572.1_2,WP_000648572.1
lcl|NZ_LOON01000008.1_cds_WP_001230983.1_3,lcl|NZ_LOON01000008.1,WP_001230983.1_3,WP_001230983.1
lcl|NZ_LOON01000008.1_cds_AUQ00_RS05235_4,lcl|NZ_LOON01000008.1,AUQ00_RS05235_4,AUQ00_RS05235
lcl|NZ_LOON01000008.1_cds_WP_000644685.1_5,lcl|NZ_LOON01000008.1,WP_000644685.1_5,WP_000644685.1
lcl|NZ_LOON01000008.1_cds_WP_001052715.1_6,lcl|NZ_LOON01000008.1,WP_001052715.1_6,WP_001052715.1


In [64]:
ToKeep.cds.host = filter(names.rem.host,Accession%in%remaining.cds.host$Accession)

In [65]:
last = anti_join(remaining.cds.host,ToKeep.cds.host,by="Accession")
last

Accession,Domain,Phylum,Class,Order,Family,Genus,Species,logFC,logCPM,PValue,FDR,EnrichedIn
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
WP_108981172.1,d__Bacteria,p__Firmicutes_A,c__Clostridia,o__Oscillospirales,f__Oscillospiraceae,g__Lawsonibacter,s__Lawsonibacter asaccharolyticus,0.8691534,10.47281,6.294604e-06,3.617103e-05,Host-associated
WP_039058451.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Kosakonia,s__Kosakonia sp000814905,0.7529128,10.44464,6.005417e-05,2.866024e-04,Host-associated
WP_109855741.1,d__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__Rhizobiales,f__Rhizobiaceae,g__Ensifer,s__Ensifer adhaerens,0.6758301,10.41614,2.441977e-04,1.021898e-03,Host-associated


#### Filter CDS of matching flagellins 

In [68]:
keep.cds.host = seqs.rem.cds.host[c(which(names(seqs.rem.cds.host) %in% ToKeep.cds.host$seqName))]

In [69]:
length(keep.cds.host)

[1] 26

In [70]:
#Merge all CDS files
all.cds.host = c(cds.host,keep.cds.host)

In [71]:
length(all.cds.host)

[1] 107

In [72]:
#Write fasta with all cds 
write.fasta(sequences=all.cds.host,names=names(all.cds.host),nbchar=80,file.out="all.cds.host.fna")

### D. Env-enriched CDS

In [73]:
seqs.rem.cds.env=read.fasta("rem.cds.env.fna")

Warning message in readLines(file):
“incomplete final line found on 'rem.cds.env.fna'”


In [74]:
names.rem.env = getSeqNames.rem("rem.cds.env.fna")

Warning message in readLines(file):
“incomplete final line found on 'rem.cds.env.fna'”


In [75]:
head(names.rem.env)

seqName,nuc,Protein,Accession
<chr>,<chr>,<chr>,<chr>
lcl|NZ_AFHI01000012.1_cds_WP_008898361.1_1,lcl|NZ_AFHI01000012.1,WP_008898361.1_1,WP_008898361.1
lcl|NZ_AFHI01000012.1_cds_WP_008898362.1_2,lcl|NZ_AFHI01000012.1,WP_008898362.1_2,WP_008898362.1
lcl|NZ_AFHI01000012.1_cds_WP_008898363.1_3,lcl|NZ_AFHI01000012.1,WP_008898363.1_3,WP_008898363.1
lcl|NZ_AFHI01000012.1_cds_WP_008898364.1_4,lcl|NZ_AFHI01000012.1,WP_008898364.1_4,WP_008898364.1
lcl|NZ_AFHI01000012.1_cds_WP_008898365.1_5,lcl|NZ_AFHI01000012.1,WP_008898365.1_5,WP_008898365.1
lcl|NZ_AFHI01000012.1_cds_WP_008898366.1_6,lcl|NZ_AFHI01000012.1,WP_008898366.1_6,WP_008898366.1


In [76]:
ToKeep.cds.env = filter(names.rem.env,Accession%in%remaining.cds.env$Accession)

In [77]:
obsolete.env = anti_join(remaining.cds.env,ToKeep.cds.env,by="Accession")
obsolete.env

Accession,Domain,Phylum,Class,Order,Family,Genus,Species,logFC,logCPM,PValue,FDR,EnrichedIn
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
WP_013189022.1,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Actinomycetales,f__Actinomycetaceae,g__Mobiluncus,s__Mobiluncus curtisii,-1.8878137,10.65105,1.808077e-30,1.484883e-28,Environmental
WP_066854468.1,d__Bacteria,p__Desulfobacterota,c__Desulfovibrionia,o__Desulfovibrionales,f__Desulfovibrionaceae,g__Halodesulfovibrio,s__Halodesulfovibrio spirochaetisodalis,-1.3894645,10.55466,9.674228e-24,3.972480e-22,Environmental
WP_029284726.1,d__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae,g__Bacillus_P,s__Bacillus_P indicus,-1.2757662,10.48323,1.792891e-22,6.425070e-21,Environmental
WP_081754523.1,d__Bacteria,p__Campylobacterota,c__Campylobacteria,o__Campylobacterales,f__Arcobacteraceae,g__Aliarcobacter,s__Aliarcobacter faecis,-1.1867007,10.69226,3.195786e-19,8.512019e-18,Environmental
WP_076920623.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Alteromonadaceae,g__Pseudoalteromonas,s__Pseudoalteromonas tetraodonis,-1.1075225,10.40328,9.538149e-13,1.323922e-11,Environmental
WP_076414249.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Burkholderiaceae,g__Achromobacter,s__Achromobacter xylosoxidans,-0.9712485,10.41003,1.221440e-12,1.631343e-11,Environmental
WP_074271829.1,d__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__Rhizobiales,f__Xanthobacteraceae,g__Bradyrhizobium,s__Bradyrhizobium erythrophlei_E,-0.8670351,10.42010,1.670043e-10,1.714403e-09,Environmental
WP_075492688.1,d__Bacteria,p__Campylobacterota,c__Campylobacteria,o__Campylobacterales,f__Campylobacteraceae,g__Campylobacter_D,s__Campylobacter_D coli,-0.8155315,10.34277,3.135319e-09,2.874285e-08,Environmental
WP_087865578.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Burkholderiaceae,g__Comamonas,s__Comamonas thiooxydans,-0.7920272,10.35090,4.594182e-09,4.134764e-08,Environmental


#### Filter CDS of matching flagellins 

In [78]:
keep.cds.env = seqs.rem.cds.env[c(which(names(seqs.rem.cds.env) %in% ToKeep.cds.env$seqName))]

In [79]:
length(keep.cds.env)

[1] 90

In [80]:
#Merge all CDS files
all.cds.env = c(cds.env,keep.cds.env)

In [81]:
length(all.cds.env)

[1] 407

In [82]:
#Write fasta with all cds 
write.fasta(sequences=all.cds.env,names=names(all.cds.env),nbchar=80,file.out="all.cds.env.fna")

# Protein sequences - filter and sort

In [83]:
#Import complete flagellin database
flagellins.markers.sb=seqinr::read.fasta("/ebio/abt3_projects/small_projects/aborbon/Thesis_FlagellinDiversity/1_shortbred/out.finalmap.faa",seqtype="AA")

### A. Animals-enriched flagellins

In [94]:
library(phylotools)

#### Sort protein fasta

In [85]:
#Table matching cds ids to protein ids
cds.to.protein.animals = getSeqNames.rem("all.cds.animals.fna")

In [86]:
#Set order of accessions
order_vec =cds.to.protein.animals$Accession

In [87]:
#Extract all cds-matching protein sequences from flagellin database
all.protein.with.cds.animals = flagellins.markers.sb[c(which(names(flagellins.markers.sb) %in% cds.to.protein.animals$Accession))]

In [88]:
#Get index for labels in order_vec
match_vec = match(names(all.protein.with.cds.animals),order_vec)

In [89]:
sorted_seqs_animals = all.protein.with.cds.animals[order(match_vec)]

In [90]:
head(sorted_seqs_animals)

$CAI29258.1
  [1] "M" "A" "Q" "V" "I" "N" "T" "N" "S" "L" "S" "L" "I" "T" "Q" "N" "N" "I"
 [19] "N" "K" "N" "Q" "S" "A" "L" "S" "S" "S" "I" "E" "R" "L" "S" "S" "G" "L"
 [37] "R" "I" "N" "S" "A" "K" "D" "D" "A" "A" "G" "Q" "A" "I" "A" "N" "R" "F"
 [55] "T" "S" "N" "I" "K" "G" "L" "T" "Q" "A" "A" "R" "N" "A" "N" "D" "G" "I"
 [73] "S" "V" "A" "Q" "T" "T" "E" "G" "A" "L" "S" "E" "I" "N" "N" "N" "L" "Q"
 [91] "R" "V" "R" "E" "L" "T" "V" "Q" "A" "T" "T" "G" "T" "N" "S" "E" "S" "D"
[109] "L" "S" "S" "I" "Q" "D" "E" "I" "K" "S" "R" "L" "D" "E" "I" "D" "R" "V"
[127] "S" "G" "Q" "T" "Q" "F" "N" "G" "V" "N" "V" "L" "A" "K" "N" "G" "S" "M"
[145] "K" "I" "Q" "V" "G" "A" "N" "D" "N" "Q" "T" "I" "T" "I" "D" "L" "K" "Q"
[163] "I" "D" "A" "K" "T" "L" "G" "L" "D" "G" "F" "S" "V" "K" "N" "N" "D" "T"
[181] "V" "T" "T" "S" "A" "P" "V" "T" "A" "F" "G" "A" "T" "T" "T" "N" "N" "I"
[199] "K" "L" "T" "G" "I" "T" "L" "S" "T" "E" "A" "A" "T" "D" "T" "G" "G" "T"
[217] "N" "P" "A" "S" "I" "E" "G" "V" "Y" "T" "D" "N

In [160]:
getwd()

[1] "/ebio/abt3_projects2/Flagellin_Diversity/code/notebooks/shortbred/edgeR/subsampled_opt1/cds"

In [159]:
#Write file
write.fasta(sequences=sorted_seqs_animals,names=names(sorted_seqs_animals),nbchar=80,file.out="all.cds.prot.animals.faa")

#### Rename CDS file

In [95]:
newLabels = select(cds.to.protein.animals,seqName,Accession)

In [104]:
#Rename using the protein accession - for selection analysis I need DNA - protein sequences with matching names
all.cds.accnLabel = rename.fasta(infile="all.cds.animals.fna",newLabels,outfile = "codon_alignments/renamed.all.cds.animals.fasta")

codon_alignments/renamed.all.cds.animals.fasta has been saved to  /ebio/abt3_projects2/Flagellin_Diversity/code/notebooks/shortbred/edgeR/subsampled_opt1/cds 


In [105]:
tmp.cds = read.fasta("codon_alignments/renamed.all.cds.animals.fasta")
match.tmp = match(names(tmp.cds),order_vec)
sorted.all.cds.protLabel.animals = tmp.cds[order(match.tmp)]
write.fasta(sequences=sorted.all.cds.protLabel.animals,names=names(sorted.all.cds.protLabel.animals),nbchar=80,file.out = "codon_alignments/sorted.cds.animals.w.protLabels.fna")

### B. Plants dataset

#### Sort protein fasta

In [106]:
#Table matching cds ids to protein ids
cds.to.protein.plants = getSeqNames.rem("all.cds.plants.fna")

In [107]:
#Extract all cds-matching protein sequences from flagellin database
all.protein.with.cds.plants = flagellins.markers.sb[c(which(names(flagellins.markers.sb) %in% cds.to.protein.plants$Accession))]

In [108]:
#Set order of accessions
order_vec.plants =cds.to.protein.plants$Accession

In [109]:
match_vec.plants = match(names(all.protein.with.cds.plants),order_vec.plants)

In [110]:
sorted_seqs_plants = all.protein.with.cds.plants[order(match_vec.plants)]

In [111]:
head(names(sorted_seqs_plants))

[1] "CAL78604.1" "ACE89738.1" "ACE93333.1" "EEJ52905.1" "ACY40759.1"
[6] "CCF11937.1"

In [112]:
#Write file
write.fasta(sequences=sorted_seqs_plants,names=names(sorted_seqs_plants),nbchar=80,file.out="all.cds.prot.plants.faa")

In [113]:
length(all.protein.with.cds.plants)

[1] 77

#### Rename CDS fasta

In [114]:
#Create matrix with oldName (col1) and newName (col2)
newLabels = select(cds.to.protein.plants,seqName,Accession)

In [115]:
head(all.cds.plants)

$`lcl|CU234118.1_cds_CAL78604.1_1`
  [1] "g" "t" "g" "c" "c" "c" "g" "c" "a" "a" "t" "c" "t" "c" "a" "a" "c" "c"
 [19] "a" "a" "c" "g" "t" "c" "g" "c" "c" "g" "c" "c" "a" "a" "c" "t" "c" "g"
 [37] "g" "c" "c" "g" "t" "c" "c" "g" "c" "t" "a" "t" "c" "t" "c" "a" "a" "c"
 [55] "a" "t" "c" "a" "a" "c" "t" "c" "c" "t" "c" "g" "c" "a" "g" "g" "a" "g"
 [73] "a" "c" "g" "a" "g" "c" "t" "c" "g" "c" "t" "c" "t" "c" "c" "a" "a" "g"
 [91] "c" "t" "g" "t" "c" "g" "a" "g" "c" "g" "g" "c" "t" "c" "g" "c" "g" "c"
[109] "a" "t" "c" "a" "c" "c" "t" "c" "c" "g" "c" "c" "t" "c" "c" "g" "a" "c"
[127] "g" "a" "c" "g" "c" "c" "g" "c" "g" "g" "g" "c" "c" "t" "c" "g" "c" "g"
[145] "a" "t" "t" "t" "c" "c" "a" "c" "c" "c" "g" "c" "a" "t" "c" "t" "c" "c"
[163] "t" "c" "c" "g" "a" "c" "a" "t" "c" "a" "c" "c" "a" "c" "g" "c" "t" "g"
[181] "c" "a" "g" "c" "a" "g" "g" "c" "g" "g" "c" "c" "a" "c" "c" "a" "a" "c"
[199] "g" "c" "g" "t" "c" "g" "c" "a" "g" "g" "c" "c" "a" "c" "t" "t" "c" "g"
[217] "a" "t" "c" "c" "t" "g"

In [116]:
#Rename using the protein accession - for selection analysis I need DNA - protein sequences with matching names
all.cds.accnLabel = rename.fasta(infile="all.cds.plants.fna",newLabels,outfile = "codon_alignments/renamed.all.cds.plants.fasta")

codon_alignments/renamed.all.cds.plants.fasta has been saved to  /ebio/abt3_projects2/Flagellin_Diversity/code/notebooks/shortbred/edgeR/subsampled_opt1/cds 


In [117]:
tmp.cds = seqinr::read.fasta("renamed.all.cds.plants.fasta")
match.tmp = match(names(tmp.cds),order_vec.plants)
sorted.all.cds.protLabel.plants = tmp.cds[order(match.tmp)]
write.fasta(sequences=sorted.all.cds.protLabel.plants,names=names(sorted.all.cds.protLabel.plants),nbchar=80,file.out = "codon_alignments/sorted.cds.plants.w.protLabels.fna")

In [118]:
head(names(sorted.all.cds.protLabel.plants))

[1] "old_name_" "old_name_" "old_name_" "old_name_" "old_name_" "old_name_"

### B. Hosts dataset

#### Sort protein fasta

In [119]:
#Table matching cds ids to protein ids
cds.to.protein.host = getSeqNames.rem("all.cds.host.fna")

In [120]:
#Extract all cds-matching protein sequences from flagellin database
all.protein.with.cds.host = flagellins.markers.sb[c(which(names(flagellins.markers.sb) %in% cds.to.protein.host$Accession))]

In [121]:
length(all.protein.with.cds.host)

[1] 107

In [122]:
#Set order of accessions
order_vec.host =cds.to.protein.host$Accession

In [123]:
match_vec.host = match(names(all.protein.with.cds.host),order_vec.host)

In [124]:
sorted_seqs_host = all.protein.with.cds.host[order(match_vec.host)]

In [125]:
head(names(sorted_seqs_host))

[1] "CAI29258.1" "EEC56699.1" "EEG95687.1" "ACR71277.1" "CBL40883.1"
[6] "EGC94816.1"

In [126]:
#Write file
write.fasta(sequences=sorted_seqs_host,names=names(sorted_seqs_host),nbchar=80,file.out="codon_alignments/all.cds.prot.host.faa")

In [127]:
length(all.protein.with.cds.host)

[1] 107

#### Rename CDS fasta

In [128]:
#Create matrix with oldName (col1) and newName (col2)
newLabels = select(cds.to.protein.host,seqName,Accession)

In [129]:
head(all.cds.host)

$`lcl|AJ865466.1_cds_CAI29258.1_1`
   [1] "a" "t" "g" "g" "c" "a" "c" "a" "a" "g" "t" "c" "a" "t" "t" "a" "a" "t"
  [19] "a" "c" "c" "a" "a" "c" "a" "g" "c" "c" "t" "c" "t" "c" "g" "c" "t" "g"
  [37] "a" "t" "c" "a" "c" "t" "c" "a" "a" "a" "a" "t" "a" "a" "t" "a" "t" "c"
  [55] "a" "a" "c" "a" "a" "g" "a" "a" "c" "c" "a" "g" "t" "c" "t" "g" "c" "g"
  [73] "c" "t" "g" "t" "c" "g" "a" "g" "t" "t" "c" "t" "a" "t" "c" "g" "a" "g"
  [91] "c" "g" "t" "c" "t" "g" "t" "c" "t" "t" "c" "t" "g" "g" "c" "t" "t" "g"
 [109] "c" "g" "t" "a" "t" "t" "a" "a" "c" "a" "g" "c" "g" "c" "g" "a" "a" "g"
 [127] "g" "a" "t" "g" "a" "c" "g" "c" "a" "g" "c" "g" "g" "g" "t" "c" "a" "g"
 [145] "g" "c" "g" "a" "t" "t" "g" "c" "t" "a" "a" "c" "c" "g" "t" "t" "t" "c"
 [163] "a" "c" "c" "t" "c" "t" "a" "a" "c" "a" "t" "t" "a" "a" "a" "g" "g" "c"
 [181] "c" "t" "g" "a" "c" "t" "c" "a" "g" "g" "c" "g" "g" "c" "c" "c" "g" "t"
 [199] "a" "a" "c" "g" "c" "c" "a" "a" "c" "g" "a" "c" "g" "g" "t" "a" "t" "c"
 [217] "t" "c" "c

In [130]:
#Rename using the protein accession - for selection analysis I need DNA - protein sequences with matching names
rename.fasta(infile="all.cds.host.fna",newLabels,outfile = "codon_alignments/renamed.all.cds.host.fasta")

codon_alignments/renamed.all.cds.host.fasta has been saved to  /ebio/abt3_projects2/Flagellin_Diversity/code/notebooks/shortbred/edgeR/subsampled_opt1/cds 


In [132]:
tmp.cds = seqinr::read.fasta("codon_alignments/renamed.all.cds.host.fasta")
match.tmp = match(names(tmp.cds),order_vec.host)
sorted.all.cds.protLabel.host = tmp.cds[order(match.tmp)]
write.fasta(sequences=sorted.all.cds.protLabel.host,names=names(sorted.all.cds.protLabel.host),nbchar=80,file.out = "codon_alignments/sorted.cds.host.w.protLabels.fna")

In [133]:
head(names(sorted.all.cds.protLabel.host))

[1] "CAI29258.1" "EEC56699.1" "EEG95687.1" "ACR71277.1" "CBL40883.1"
[6] "EGC94816.1"

In [134]:
nrow(newLabels)

[1] 107

In [135]:
head(names(sorted_seqs_host))

[1] "CAI29258.1" "EEC56699.1" "EEG95687.1" "ACR71277.1" "CBL40883.1"
[6] "EGC94816.1"

### B. Env-enriched flagellins

#### Sort protein fasta

In [136]:
#Table matching cds ids to protein ids
cds.to.protein.env = getSeqNames.rem("all.cds.env.fna")

In [137]:
#Extract all cds-matching protein sequences from flagellin database
all.protein.with.cds.env = flagellins.markers.sb[c(which(names(flagellins.markers.sb) %in% cds.to.protein.env$Accession))]

In [138]:
length(all.protein.with.cds.env)

[1] 405

In [139]:
#Set order of accessions
order_vec.env =cds.to.protein.env$Accession

In [140]:
match_vec.env = match(names(all.protein.with.cds.env),order_vec.env)

In [141]:
sorted_seqs_env = all.protein.with.cds.env[order(match_vec.env)]

In [142]:
head(names(sorted_seqs_env))

[1] "AAA62844.1" "CAA59172.1" "AAK24938.1" "AAR10729.1" "BAD14977.1"
[6] "AAW75835.1"

In [143]:
#Write file
write.fasta(sequences=sorted_seqs_env,names=names(sorted_seqs_env),nbchar=80,file.out="codon_alignments/all.cds.prot.env.faa")

In [144]:
length(all.protein.with.cds.env)

[1] 405

#### Rename CDS fasta

In [145]:
#Create matrix with oldName (col1) and newName (col2)
newLabels.env = select(cds.to.protein.env,seqName,Accession)

In [146]:
head(all.cds.env)

$`lcl|L15367.1_cds_AAA62844.1_1`
   [1] "a" "t" "g" "g" "c" "t" "t" "t" "a" "a" "c" "t" "g" "t" "a" "a" "a" "c"
  [19] "a" "c" "c" "a" "a" "c" "a" "t" "t" "g" "c" "g" "t" "c" "g" "a" "t" "c"
  [37] "a" "c" "t" "a" "c" "t" "c" "a" "g" "g" "g" "c" "a" "a" "c" "c" "t" "g"
  [55] "a" "c" "c" "a" "a" "g" "g" "c" "t" "a" "g" "c" "a" "a" "c" "g" "c" "t"
  [73] "c" "a" "g" "a" "c" "c" "a" "c" "t" "t" "c" "g" "a" "t" "g" "c" "a" "g"
  [91] "c" "g" "t" "c" "t" "g" "t" "c" "c" "t" "c" "g" "g" "g" "t" "c" "t" "g"
 [109] "c" "g" "t" "a" "t" "c" "a" "a" "c" "a" "g" "c" "g" "c" "t" "a" "a" "a"
 [127] "g" "a" "c" "g" "a" "c" "g" "c" "t" "g" "c" "c" "g" "g" "c" "c" "t" "g"
 [145] "c" "a" "g" "a" "t" "c" "g" "c" "c" "a" "a" "c" "c" "g" "t" "c" "t" "g"
 [163] "a" "c" "c" "a" "g" "c" "c" "a" "g" "a" "t" "c" "a" "a" "t" "g" "g" "c"
 [181] "c" "t" "g" "g" "g" "c" "c" "a" "g" "g" "c" "t" "g" "t" "c" "a" "a" "g"
 [199] "a" "a" "c" "g" "c" "c" "a" "a" "c" "g" "a" "c" "g" "g" "t" "a" "t" "c"
 [217] "t" "c" "g" 

In [147]:
#Rename using the protein accession - for selection analysis I need DNA - protein sequences with matching names
rename.fasta(infile="all.cds.env.fna",newLabels.env,outfile = "codon_alignments/renamed.all.cds.env.fasta")

codon_alignments/renamed.all.cds.env.fasta has been saved to  /ebio/abt3_projects2/Flagellin_Diversity/code/notebooks/shortbred/edgeR/subsampled_opt1/cds 


In [148]:
length(match_vec.env)

[1] 405

In [151]:
tmp.cds = seqinr::read.fasta("codon_alignments/renamed.all.cds.env.fasta")
match.tmp = match(names(tmp.cds),order_vec.host)
sorted.all.cds.protLabel.env = tmp.cds[order(match.tmp)]
write.fasta(sequences=sorted.all.cds.protLabel.env,names=names(sorted.all.cds.protLabel.env),nbchar=80,file.out = "codon_alignments/sorted.cds.env.w.protLabels.fna")

In [152]:
head(names(sorted.all.cds.protLabel.env))

[1] "EAP91499.1" "EDK26613.1" "BAD14977.1" "EDZ58145.1" "EEJ52905.1"
[6] "EEO87047.1"

In [153]:
length(sorted.all.cds.protLabel.env)

[1] 407

In [154]:
nrow(newLabels)

[1] 107

In [155]:
head(names(sorted_seqs_env))

[1] "AAA62844.1" "CAA59172.1" "AAK24938.1" "AAR10729.1" "BAD14977.1"
[6] "AAW75835.1"

# Sort alignments

#### Animals

In [158]:
getwd()

[1] "/ebio/abt3_projects2/Flagellin_Diversity/code/notebooks/shortbred/edgeR/subsampled_opt1/cds"

In [157]:
aln.animals = read.fasta("codon_alignments/alignment.all.cds.prot.animals.fasta")

Warning message in file(con, "r"):
“cannot open file 'codon_alignments/alignment.all.cds.prot.animals.fasta': No such file or directory”


ERROR: Error in file(con, "r"): cannot open the connection


In [ ]:
head(names(aln.animals))

In [ ]:
match_vec.aln = match(names(aln.animals),order_vec)

In [ ]:
sorted_alignment = aln.animals[order(match_vec.aln)]

In [ ]:
head(names(sorted_alignment))

In [ ]:
write.fasta(sequences=sorted_alignment,names=names(sorted_alignment),nbchar=80,file.out="codon_alignments/sorted_alignment.prot.animals.fasta")

#### Plants

In [ ]:
aln.plants = read.fasta("codon_alignments/alignment.all.cds.prot.plants.fasta")

In [ ]:
head(names(aln.plants))

In [ ]:
match_vec.aln.pl = match(names(aln.plants),order_vec.plants)

In [ ]:
sorted_alignment.pl = aln.plants[order(match_vec.aln.pl)]

In [ ]:
head(names(sorted_alignment.pl))

In [ ]:
write.fasta(sequences=sorted_alignment.pl,names=names(sorted_alignment.pl),nbchar=80,file.out="codon_alignments/sorted_alignment.prot.plants.fasta")

#### Host

In [ ]:
aln.host = read.fasta("codon_alignments/alignment.all.cds.prot.host.fasta")

In [ ]:
head(names(aln.host))

In [ ]:
match_vec.aln.h = match(names(aln.host),order_vec.host)

In [ ]:
sorted_alignment.h = aln.host[order(match_vec.aln.h)]

In [ ]:
head(names(sorted_alignment.h))

In [ ]:
write.fasta(sequences=sorted_alignment.h,names=names(sorted_alignment.h),nbchar=80,file.out="codon_alignments/sorted_alignment.prot.host.fasta")

#### Environmental

In [ ]:
aln.env = read.fasta("codon_alignments/alignment.all.cds.prot.env.fasta")

In [ ]:
head(names(aln.env))

In [ ]:
match_vec.aln.env = match(names(aln.env),order_vec.env)

In [ ]:
sorted_alignment.env = aln.env[order(match_vec.aln.env)]

In [ ]:
head(names(sorted_alignment.env))

In [ ]:
length(sorted_alignment.env)

In [ ]:
write.fasta(sequences=sorted_alignment.env,names=names(sorted_alignment.env),nbchar=80,file.out="codon_alignments/sorted_alignment.prot.env.fasta")

# Split animals_plants codon alignments

In [162]:
paml.plants.animals = read.fasta("codon_alignments/paml_aln_plants.animals.fasta")

In [168]:
length(paml.plants.animals)

[1] 192

In [177]:
paml.animals = paml.plants.animals[c(which(names(paml.plants.animals) %in% cds.to.protein.animals$Accession))]

In [178]:
paml.plants = paml.plants.animals[c(which(names(paml.plants.animals) %in% cds.to.protein.plants$Accession))]

In [180]:
write.fasta(sequences=paml.animals,names=names(paml.animals),nbchar=80,file.out="codon_alignments/paml.animals.fasta")
write.fasta(sequences=paml.plants,names=names(paml.plants),nbchar=80,file.out="codon_alignments/paml.plants.fasta")

In [166]:
getwd()

[1] "/ebio/abt3_projects2/Flagellin_Diversity/code/notebooks/shortbred/edgeR/subsampled_opt1/cds"